# Employee Information System - Before MCP Implementation

This notebook demonstrates a traditional approach to creating an agent that handles employee information queries before implementing the Model Context Protocol (MCP). We'll create custom tools directly within the notebook and use them with a language model to answer questions about employees.

## Key Components

1. **Azure OpenAI Integration**: Using Azure's OpenAI service to power our agent
2. **Custom Tool Definitions**: Creating tools for employee information retrieval directly in the notebook
3. **Agent Creation**: Building a ReAct agent with the defined tools
4. **Query Execution**: Testing the agent with employee information requests

## Required Library Imports

The following cell imports all necessary libraries for working with Azure OpenAI, LangGraph, and creating custom tools.

In [ ]:
from langchain_openai import AzureChatOpenAI
import random
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
import time

In [ ]:
AZURE_API_VERSION = "2024-10-21"
AZURE_ENDPOINT = "Azure_End_Point"
AZURE_API_KEY = "<Azure_API_KEy>"

def llm():
    model = AzureChatOpenAI(
        api_version= AZURE_API_VERSION,
        azure_endpoint= AZURE_ENDPOINT,
        api_key= AZURE_API_KEY,
        azure_deployment="gpt-4o",
        verbose=True
    )
    return model

azure_api_client = llm()

## Azure OpenAI Configuration

Setting up the Azure OpenAI client with appropriate API credentials and parameters. This model will serve as the brain of our agent.

## Custom Tool Definitions

In this section, we define several custom tools that our agent will use to retrieve employee information. Each tool is decorated with the `@tool` decorator to make it compatible with the LangChain framework.

The tools we're defining include:
- **get_employee_supervisor**: Retrieves the supervisor for a given employee ID
- **get_employee_location**: Retrieves the location where an employee is based
- **get_employee_ID**: Retrieves the employee ID for a given name
- **get_employee_skill_set**: Retrieves the primary skill for a given employee ID

Note: For demonstration purposes, these tools return random values rather than querying a real database.

In [ ]:
@tool
def get_employee_supervisor(employee_ID: str):
    """
    Returns the supervisor for a given employee ID.

    Args:
        employee_ID (str): The unique identifier of an employee.

    Returns:
        str: Supervisor's name associated with the given employee ID.
    """
    list_of_supervisors = ['Michael', 'Jessica', 'David', 'Ashley', 'Christopher']
    supervisor = random.choice(list_of_supervisors)
    return f"The supervisor for the given employee is {supervisor}"

@tool
def get_employee_location(employee_name: str):
    """
    Returns the location of a given employee.

    Args:
        employee_name (str): The name of the employee.

    Returns:
        str: Location where the employee is based.
    """
    list_of_locations = ["Hyderbad", "Bangalore", "Chennai", "Mumbai"]
    location = random.choice(list_of_locations)
    return f"The location for {employee_name} is {location}"

@tool
def get_employee_ID(employee_name: str):
    """
    Returns the employee ID for a given employee name.

    Args:
        employee_name (str): The name of the employee.

    Returns:
        str: Unique ID assigned to the employee.
    """
    # Note: This tool has an intentional 30-second delay to simulate a slow database query
    # In real applications, this delay might represent a complex database lookup or external API call
    print("seelping for 30 seconds")
    time.sleep(30)
    list_of_ids = ['abd104', '3ni3n', '93jnj', 'ikh2k']
    employee_ID = random.choice(list_of_ids)
    return employee_ID

@tool
def get_employee_skill_set(employee_ID: str):
    """
    Returns the skill set of a given employee ID.

    Args:
        employee_ID (str): The unique identifier of the employee.

    Returns:
        str: Primary skill associated with the employee.
    """
    skill = random.choice(["Machine Learning", "Generative AI", "ML Ops", "Image Analysis"])
    return skill


In [4]:
agent_system_prompt = """
You are an assistant designed to provide employee-related information using the available tools.
"""


## Agent System Prompt

The system prompt provides context and instructions to the language model about its role. Here, we're defining a simple prompt that tells the model it's an assistant for employee information.

In [ ]:
employee_info_agent = create_react_agent(
    model= llm(),  
    tools=[get_employee_supervisor, get_employee_location, get_employee_ID,get_employee_skill_set],  
    prompt= agent_system_prompt,
    checkpointer= InMemorySaver()
)
# Creating the agent with our custom tools and configuring checkpointing for session persistence

## Creating the Agent

Now we'll create a ReAct agent that combines the Azure OpenAI model with our custom tools. This agent will be able to understand natural language queries and decide which tools to use to answer them.

We also set up an InMemorySaver for checkpointing, which allows the agent to maintain state between interactions.

In [ ]:
user_query = "what is the location of leo?"
session_id = "abc_7"
thread_config = {
    "configurable": {"thread_id": session_id}
}
User_query_prompt = {
    "messages": [{"role": "user", "content": user_query}]
    }
# Setting up a user query and session ID for continuity across interactions

## Executing the Query

Finally, we'll invoke the agent with our user query and session configuration. The agent will determine which tools to use and provide a response based on the information it retrieves.

In [ ]:
# Execute the query and return the agent's response
# Note: The agent will decide which tools to use based on the query
employee_info_agent.invoke(User_query_prompt,config= thread_config )

## Try Different Queries

You can modify the user query in the cell above to ask different questions about employees. For example:

- "Who is the supervisor for employee with ID abd104?"
- "What are the skills of the employee with ID 3ni3n?"
- "Get me the employee ID for Jessica"

## Limitations of This Approach

This approach has several limitations compared to the MCP implementation:

1. **Tight Coupling**: Tools are directly defined in the notebook, making them difficult to maintain or update independently
2. **Limited Scalability**: Adding new tools requires modifying this notebook and restarting the agent
3. **No Service Architecture**: Tools can't easily be deployed as separate services or shared across different agents
4. **Performance Issues**: As seen with the intentional delay in `get_employee_ID`, performance issues in one tool affect the entire system

In the MCP implementation, these limitations are addressed by separating tools into dedicated services that can be independently deployed, scaled, and maintained.

## Conclusion

In this notebook, we've demonstrated how to create a basic employee information system using custom tools defined directly within the notebook. While this approach works for simple cases, it has significant limitations in terms of scalability, maintainability, and performance isolation.

The next step is to refactor this system using the Model Context Protocol (MCP), which will allow us to separate the tools into dedicated services that can be independently deployed and maintained. This approach is demonstrated in the `after_mcp.ipynb` notebook.

## References

1. [LangChain Tool Decorators](https://python.langchain.com/docs/modules/agents/tools/custom_tools)
2. [LangGraph ReAct Agents](https://python.langchain.com/docs/templates/react)
3. [Model Context Protocol Documentation](https://modelcontextprotocol.io/docs/concepts/tools)